<a href="https://colab.research.google.com/github/solobala/DLL30/blob/main/DLL30_DZ6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание по теме «Рекуррентные сети»

Преподаватель: Даниил Корбут, Алексей Миронов, Татьяна Булгакова, Светлана Медведева, Артём Качалкин, Никита Селин


# Задание 1. Обучите нейронную сеть решать шифр Цезаря

# Импорт библиотек и пользовательские функции

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
from google.colab import files

In [3]:
from time import time
import math
import string
import random
from tqdm.notebook import tqdm
from pathlib import Path
import re

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Параметры обучения
K = 10 # максимальный сдвиг. В этом случае accuracy не превышает 50%, даже если запихнуть сдвиг в параметры модели и подготовить большой обучающий датасет  спримерами с разными сдвигами

SEED = 1234
BATCH_SIZE = 50
random.seed(SEED)
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device=='cpu':
  torch.manual_seed(SEED)
else:
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

In [8]:
device

device(type='cuda')

In [9]:
all_letters='абвгдежзийклмнопрстуфхцчшщъыьэюя '
CHARS = list(all_letters)  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR =[w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс


In [10]:
INDEX_TO_CHAR

['а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 ' ']

In [11]:
CHAR_TO_INDEX

{'а': 0,
 'б': 1,
 'в': 2,
 'г': 3,
 'д': 4,
 'е': 5,
 'ж': 6,
 'з': 7,
 'и': 8,
 'й': 9,
 'к': 10,
 'л': 11,
 'м': 12,
 'н': 13,
 'о': 14,
 'п': 15,
 'р': 16,
 'с': 17,
 'т': 18,
 'у': 19,
 'ф': 20,
 'х': 21,
 'ц': 22,
 'ч': 23,
 'ш': 24,
 'щ': 25,
 'ъ': 26,
 'ы': 27,
 'ь': 28,
 'э': 29,
 'ю': 30,
 'я': 31,
 ' ': 32}

In [12]:
n_letters = len(all_letters)
n_letters

33

In [13]:
len(CHARS)

33


**Алгоритм шифра Цезаря для генерации выборки:**

1\. Создаем словарь, где ключами будут буквы алфавита, а значениями \- их порядковые номера \(например, \{'А': 0, 'Б': 1, 'В': 2, \.\.\.\}\)

2\. Создаем функцию для шифрования текста с помощью шифра Цезаря\. Функция принимает на вход текст и параметр К \(сдвиг\), и возвращает зашифрованный текст\. Для этого мы проходим по каждой букве в тексте и заменяем ее на букву, которая находится на К позиций правее в алфавите\. Если при этом мы выходим за пределы алфавита, то начинаем заново с первой буквы \(например, при К\=2 буква “Я” заменяется на букву “А”\)\.

3\. Создаем функцию для генерации выборки\. Функция принимает на вход количество примеров \(N\) и максимальный сдвиг \(K\)\. Затем мы генерируем N случайных строк длиной от 5 до 20 символов, выбираем для каждой строки случайный сдвиг от 1 до K и шифруем ее с помощью функции из пункта 2\. В качестве ответа сохраняем дешифрованную строку\.





In [14]:
# Функция шифрования текста с помощью шифра Цезаря
def caesar_encrypt(text, k):
    """функция для шифрования текста с помощью шифра Цезаря.
    Функция принимает на вход текст и параметр К (сдвиг), и возвращает зашифрованный текст"""
    cipher_text = ''
    for letter in text:
        if letter in CHARS:
            index = (CHARS.index(letter) + k) % len(CHARS)
            cipher_text += CHARS[index]
        else:
            cipher_text += letter
    return cipher_text

In [15]:
caesar_encrypt('я', 3) # проверка

'б'

In [16]:
def caesar_decrypt(ciphertext, shift):
    """
    Функция  дешифровки - справочно.
    """
    plaintext = ""
    for char in ciphertext:
      i = INDEX_TO_CHAR.index(char)
      j = i - shift
      if j < 0:
        j +=33
      decrypted_char = INDEX_TO_CHAR[j]
      plaintext += decrypted_char
    return plaintext

In [17]:
caesar_decrypt('б', 3) # проверка

'я'

# Загрузка данных

In [18]:
# для обучения модели используем  книгу Ольги Громыко "Киборг и его лесник"
drive.mount('/content/drive')
with open(Path('/content/drive/MyDrive/Gromyko_Kosmooluhi_6.txt'), 'r', encoding="utf-8") as f1:
    all_text = f1.read().lower()


Mounted at /content/drive


In [19]:
all_text[:350]

'ольга громыко\n\nкиборг и его лесник\n\n«уважаемый евгений николаевич!\n\nфинансовое управление североомлошского лесничества сообщает, что в настоящее время выделить денежные средства на покупку служебного киборга не представляется возможным в связи с отсутствием свободных денежных средств.\n\nзаместитель главы сол полещук григорий борисович».\n\nвнизу было '

In [20]:
# Делим текст на строки, удаляем пустые строки, обрезаем каждое предложение до MAX_LEN символов
plain_sentences = all_text.split('\n')


In [21]:
# Удаляем все символы, кроме кириллицы и пробелов
pattern = re.compile('[^а-я\s]+')
for i in range(len(plain_sentences)):
    plain_sentences[i] = re.sub(pattern, '', plain_sentences[i])
    plain_sentences[i].strip()

In [22]:
# Заеменяем несколько подряд стоящих пробелов на 1
pattern = re.compile('[\s\s]+')
for i in range(len(plain_sentences)):
    plain_sentences[i] = re.sub(pattern, ' ', plain_sentences[i])
    plain_sentences[i].strip()

In [23]:
plain_sentences = [x[:MAX_LEN] for x in plain_sentences if x != '']

In [ ]:
# # Добавляем в конец пробелы для достижения строкой длины MAX_LEN
# plain_sentences = [x + ' ' * (MAX_LEN - len(x)) for x in plain_sentences if len(x) < MAX_LEN]

In [24]:
for i in range(len(plain_sentences)):
    plain_sentences[i].strip()

In [25]:
plain_sentences[8:20]

['насквозь промокший с раскалывающейся башкой злой к',
 'уволюсь злобно подумал женька уволюсь к чертовой м',
 'башка до сих пор гудела в месте удара прощупывалос',
 'поплескав в лицо холодной водой и немного очухавши',
 'снаружи уже рассвело хотя ленивое эдемское солнце ',
 'если конечно у них не было киборгов',
 'пойду на обход может развеюсь решил женька вытаски',
 'женька покосился на холодильник обход мог растянут',
 'сунув в кобуру станнер а в ножны профессиональный ',
 'женька напоследок окинул взглядом свое скромное хо',
 'в десяти метрах от него стоял матерый двухсоткилог',
 'несколько секунд зверь и лесник буравили друг друг']

In [26]:
train_plain_sentences, test_plain_sentences = train_test_split(plain_sentences, test_size=0.2, random_state=random.seed(SEED))

In [27]:
def make_cypher_dict(plain_sentences):
  # Готовим датасет для обучения модели. Сдвиг - в границах от 1 до 9 включительно,
  # поэтому словарь зашифрованных строк будет иметь ключи от 1 до 9
  # и значения - строки, зашифрованные на этих ключах
  cypher_sentences = dict()
  v1, v2, v3, v4, v5, v6, v7, v8, v9 = [],[],[],[],[],[],[],[],[]
  for item in plain_sentences:
    # k = random.randint(1, K) # Закомментировано для проверки, измениться ли loss и accuracy для единственного значения сдвига
    k = 2
    cipher_item = caesar_encrypt(item, k)
    if k == 1:
        v1.append(cipher_item)
    elif k == 2:
        v2.append(cipher_item)
    elif k == 3:
        v3.append(cipher_item)
    elif k == 4:
        v4.append(cipher_item)
    elif k == 5:
        v5.append(cipher_item)
    elif k == 6:
        v6.append(cipher_item)
    elif k == 7:
        v7.append(cipher_item)
    elif k == 8:
        v8.append(cipher_item)
    elif k == 9:
        v9.append(cipher_item)
  cypher_sentences["1"] = v1
  cypher_sentences["2"] = v2
  cypher_sentences["3"] = v3
  cypher_sentences["4"] = v4
  cypher_sentences["5"] = v5
  cypher_sentences["6"] = v6
  cypher_sentences["7"] = v7
  cypher_sentences["8"] = v8
  cypher_sentences["9"] = v9
  return cypher_sentences

In [28]:
train_cypher_sentences = make_cypher_dict(train_plain_sentences)
test_cypher_sentences = make_cypher_dict(test_plain_sentences)

In [29]:
train_cypher_sentences["2"][8:15]

['узержпабнзупкмбстрнрикнбовтътхфбсрбрсхъмвобджрнюбц',
 'унзжх ыкобтэдмробнзупкмвбдйжзтпхнкбпвбпрекбфхфбизб',
 'фрфбрмвйвнуабпвбсртажрмбунрипззбусхумвбтхмкбкбпрек',
 'жвнюпзлъззбтвуунзжрдвпкзбсрмвйвнрбщфрбудрнрщюбстръ',
 'сткънруюбутрщпрбргртвщкдвфюуабмбозпюъзохбйнхбкбжзн',
 'бмхскбпрдх бумрдртржмхбхбяфрлбснрчрзбфзтортвустзжз',
 'бфзгзбяфрбпкщзербпзбпвсрокпвзф']

In [30]:
# Убедимся, что в датасете примерно равное к-во строк, зашифрованных на разных ключах
for key, value in train_cypher_sentences.items():
  print (key, len(value))

1 0
2 4124
3 0
4 0
5 0
6 0
7 0
8 0
9 0


In [31]:
len(plain_sentences)

5156

In [33]:
# all_categories =[str(i) for i in range(1,10)]
all_categories = ['2']
all_categories

['2']

In [34]:
n_categories = len(all_categories)

In [35]:
# Особенность модели - и в скрытое состояние, и в output добавлен вектор категорий
class NameGeneratorModule(nn.Module):
    def __init__(self, inp_size, hid_size, op_size):
        super(NameGeneratorModule, self).__init__()
        self.hid_size = hid_size
        self.inp_size = inp_size
        self.op_size = op_size

        self.i2h = nn.Linear(n_categories + inp_size + hid_size, hid_size)
        self.i2o = nn.Linear(n_categories + inp_size + hid_size, op_size)
        self.o2o = nn.Linear(hid_size + op_size, op_size)
        self.dropout = nn.Dropout(0.1)
        # self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        inp_comb = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(inp_comb)
        output = self.i2o(inp_comb)
        op_comb = torch.cat((hidden, output), 1)
        output = self.o2o(op_comb)
        output = self.dropout(output)
        # output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hid_size)

In [36]:
# Выюираем случайный элемент списка
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

In [62]:
all_categories.index('2')

0

In [65]:
# Выбираем случайную категорию ( сдвиг от 1 до 9 включительно)
# случайную зашифрованную строку для этой категории
# и получаем расшифровку
def randomPair(data):
    category = randomChoice(all_categories)
    line = randomChoice(data[category])
    plain = caesar_decrypt(line, int(category))
    # plain = caesar_decrypt(line, all_categories.index(category)+1)
    return category, line, plain

In [66]:
category, line, plain = randomPair(train_cypher_sentences)
category, line, plain

('2',
 'нзупкмбсрмрукнуабпвбпзербкбдпзйвспрбсржховнбвбдзжю',
 'лесник покосился на него и внезапно подумал а ведь')

In [84]:
# One-hot вектор длиной n_categories, состоящий из 0, 1 - на месте выбранной категории
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

In [85]:
categoryTensor(category), categoryTensor(category).shape

(tensor([[1.]]), torch.Size([1, 1]))

In [86]:
# One-hot матрица с 1 до предпоследней буквы строки для input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

In [87]:
inputTensor(line), inputTensor(line).shape

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 1., 0., 0.]]]),
 torch.Size([50, 1, 33]))

In [88]:
# target тензор - это расшифрованное слово
plain = caesar_decrypt(line, int(category))
plain, len(plain)

('лесник покосился на него и внезапно подумал а ведь', 50)

In [89]:
def targetTensor(plain):
    letter_indexes = [CHAR_TO_INDEX[plain[li]] for li in range(len(plain))]
    # letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [90]:
target = targetTensor(plain)
target, len(target)

(tensor([11,  5, 17, 13,  8, 10, 32, 15, 14, 10, 14, 17,  8, 11, 17, 31, 32, 13,
          0, 32, 13,  5,  3, 14, 32,  8, 32,  2, 13,  5,  7,  0, 15, 13, 14, 32,
         15, 14,  4, 19, 12,  0, 11, 32,  0, 32,  2,  5,  4, 28]),
 50)

In [91]:
def timeSince(since):
    now = time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [92]:
# Определяем колбэк ModelCheckpoint для сохранения параметров модели с наименьшей loss
class ModelCheckpoint:
    def __init__(self, filepath, save_top_k=1, monitor='loss', mode='min'):
        self.filepath = filepath
        self.save_top_k = save_top_k
        self.monitor = monitor
        self.mode = mode
        self.best_score = None
        self.best_model = None
        self.top_k_models = []

    def __call__(self, iter, model, loss):
        if self.best_score is None or (loss < self.best_score and self.mode == 'min') or (loss > self.best_score and self.mode == 'max'):
            self.best_score = loss
            self.best_model = model.state_dict()
            if len(self.top_k_models) < self.save_top_k:
                self.top_k_models.append((self.best_score, self.best_model))
            else:
                self.top_k_models.sort(key=lambda x: x[0], reverse=self.mode == 'max')
                worst_model_score, worst_model_state_dict = self.top_k_models[-1]
                if (worst_model_score < loss and self.mode == 'min') or (worst_model_score > loss and self.mode == 'max'):
                    self.top_k_models[-1] = (loss, model.state_dict())
            torch.save(self.best_model, self.filepath)

In [93]:
# Обучаем модель с использованием колбэка ModelCheckpoint
checkpoint_callback = ModelCheckpoint(filepath='best_model.pth', save_top_k=1, monitor='loss', mode='min')
callbacks = [checkpoint_callback]

In [103]:
learning_rate = 0.001
num_epochs = 100000
model = NameGeneratorModule(n_letters, 256, n_letters)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# Loss
criterion = torch.nn.CrossEntropyLoss()
lr = 0.00005

In [95]:
print(model)

NameGeneratorModule(
  (i2h): Linear(in_features=290, out_features=256, bias=True)
  (i2o): Linear(in_features=290, out_features=33, bias=True)
  (o2o): Linear(in_features=289, out_features=33, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [ ]:
p = torch.tensor([[-0.0503,  0.0786, -0.0593, -0.0999,  0.0994, -0.0180,  0.0909, -0.0097,
           0.0179, -0.0263, -0.0000, -0.0514, -0.0303,  0.0000, -0.0080, -0.0019,
           0.0587,  0.0251, -0.0933,  0.1296,  0.1146, -0.0900, -0.0175, -0.0360,
          -0.0742, -0.0209, -0.1497, -0.0000,  0.1171,  0.0708, -0.0310,  0.0269,
          -0.0822]])
values, indices =  p.topk(1)
indices.flatten().numpy()[0]

19

In [96]:
def count_matching_values(c, output):
    matches = torch.eq(c, output)
    count = torch.sum(matches).item()
    return count

In [97]:
def eval(category, line, plain):
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(plain)
    target_line_tensor.unsqueeze_(-1)
    hidden = model.initHidden()
    model.eval()
    val_loss = 0
    output = None
    with torch.no_grad():
      # Инициируем сгенерированную расшифровку как тензор из 0 ( далее там будут индексы букв)
      generated = torch.zeros(len(plain), 1, dtype=int)
      if len(line) > 0:
          for i in range(len(line)):
              output, hidden = model(category_tensor, input_line_tensor[i], hidden)
              values, indices = output.topk(1)
              generated[i] = indices.flatten().numpy()[0]
              l = criterion(output, target_line_tensor[i])
              val_loss += l
          val_loss = val_loss.item() / input_line_tensor.size(0)
      else:
          pass
      test_acc = count_matching_values(target_line_tensor, generated)/len(line)
    return output, val_loss, test_acc

In [98]:
def train(category, line, plain):
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(plain)
    target_line_tensor.unsqueeze_(-1)
    hidden = model.initHidden()
    model.train()
    optimizer.zero_grad()
    loss = 0
    output = None
    # Инициируем сгенерированную расшифровку как тензор из 0 ( далее там будут индексы букв)
    generated = torch.zeros(len(plain), 1, dtype=int)
    if len(line) > 0:
        for i in range(len(line)):
            output, hidden = model(category_tensor, input_line_tensor[i], hidden)
            values, indices = output.topk(1)
            generated[i] = indices.flatten().numpy()[0]
            l = criterion(output, target_line_tensor[i])
            loss += l
        if loss != 0:
          loss.backward(retain_graph=True)
          optimizer.step()
        loss = loss.item() / input_line_tensor.size(0)
        # for p in model.parameters():
        #   p.data.add_(p.grad.data, alpha=-learning_rate)

    else:
        pass
    train_acc = count_matching_values(target_line_tensor, generated)/len(line)
    return output, loss, train_acc

In [99]:
a,b,c = randomPair(train_cypher_sentences)

In [100]:
train(a, b, c)

(tensor([[ 0.0003,  0.0696,  0.0324, -0.0840, -0.0407,  0.0574, -0.0328, -0.0177,
           0.0000, -0.0393, -0.1190,  0.0271,  0.0205, -0.0834, -0.0750, -0.0630,
           0.1288,  0.1067,  0.0144,  0.0089, -0.0132,  0.0204,  0.0607, -0.0378,
          -0.0477, -0.0000, -0.1204, -0.0006, -0.1002,  0.0665,  0.0000,  0.0804,
          -0.0137]], grad_fn=<MulBackward0>),
 3.5126797485351564,
 0.0)

In [101]:
d,e,f = randomPair(test_cypher_sentences)

In [102]:
eval(d,e,f)

(tensor([[ 0.0020,  0.0263, -0.0222, -0.0836, -0.1161,  0.0906, -0.0729,  0.0255,
           0.0664, -0.0719, -0.0800, -0.0638, -0.0504,  0.0147, -0.0664, -0.0346,
           0.0203,  0.1059,  0.0156,  0.0100, -0.0037,  0.0137,  0.0287,  0.0027,
          -0.0600, -0.0424, -0.1243, -0.0624, -0.1090,  0.0565,  0.0207,  0.0121,
           0.0444]]),
 3.4647052001953127,
 0.08)

In [104]:
total_loss = 0 # Reset every plot_every iters
total_val_loss = 0
best_loss = 10.
best_epoch = 0
print_every = 5000
train_acc, total_train_acc, total_val_acc = 0., 0., 0.
start = time()
for iter in tqdm(range(1, num_epochs + 1)):

    # training loop
    a, b, c = randomPair(train_cypher_sentences)
    train_output, loss, train_acc = train(a, b, c)
    total_train_acc += train_acc
    if loss is not None:
        if best_loss > loss:
          best_loss = loss
          best_epoch = iter
        total_loss += loss

    # evaluation loop
    d,e,f = randomPair(test_cypher_sentences)
    test_output, val_loss, val_acc = eval(d,e,f)
    total_val_acc += val_acc
    if val_loss is not None:
      total_val_loss += val_loss

    if iter % print_every == 0:
      print('Time: %s, Epoch: (%d - Total Iterations: %d%%),  Loss: %.4f, Train Accuracy: %.4f, val Loss: %.4f, Val Accuracy: %.4f' % (timeSince(start), iter, iter / num_epochs * 100, total_loss / iter, total_train_acc/iter,  total_val_loss / iter, total_val_acc/iter))

    # Вызов колбэков
    for callback in callbacks:
      callback(iter, model, loss)


  0%|          | 0/100000 [00:00<?, ?it/s]

Time: 3m 3s, Epoch: (5000 - Total Iterations: 5%),  Loss: 271919.8875, Train Accuracy: 0.8138, val Loss: 350943.4354, Val Accuracy: 0.8792
Time: 6m 8s, Epoch: (10000 - Total Iterations: 10%),  Loss: 135998.6147, Train Accuracy: 0.6336, val Loss: 175504.6235, Val Accuracy: 0.6911
Time: 9m 9s, Epoch: (15000 - Total Iterations: 15%),  Loss: 90666.2045, Train Accuracy: 0.7063, val Loss: 117003.2955, Val Accuracy: 0.7727
Time: 12m 12s, Epoch: (20000 - Total Iterations: 20%),  Loss: 67999.7032, Train Accuracy: 0.7603, val Loss: 87752.4749, Val Accuracy: 0.8287
Time: 15m 11s, Epoch: (25000 - Total Iterations: 25%),  Loss: 54399.7902, Train Accuracy: 0.7943, val Loss: 70201.9800, Val Accuracy: 0.8630
Time: 18m 9s, Epoch: (30000 - Total Iterations: 30%),  Loss: 45333.1811, Train Accuracy: 0.8170, val Loss: 58501.6500, Val Accuracy: 0.8858
Time: 21m 8s, Epoch: (35000 - Total Iterations: 35%),  Loss: 38857.0317, Train Accuracy: 0.8331, val Loss: 50144.2714, Val Accuracy: 0.9021
Time: 24m 6s, Epoc

! Проверить: Если взять только 1 цифру для сдвига и переделать трейновые  и тестовые данные, Accuracy существенно вырастает, а loss падает

In [105]:
best_epoch, best_loss

(3775, 0.0)

In [106]:
best_model = NameGeneratorModule(n_letters, 256, n_letters)
best_model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [107]:
best_model.eval()


NameGeneratorModule(
  (i2h): Linear(in_features=290, out_features=256, bias=True)
  (i2o): Linear(in_features=290, out_features=33, bias=True)
  (o2o): Linear(in_features=289, out_features=33, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [130]:
def generate_text(sentence):
  all_indices = []
  best_model.eval()

  # Для каждой категории
  for category in all_categories:
    category_tensor = categoryTensor(category) # Тензор для текущей категории
    hidden = model.initHidden() # Инициируем скрытое состояние

    # для каждой буквы из шифровки
    for i in range(len(sentence)):
      s = sentence[i]
      input_line_tensor = inputTensor(s)

      # Передаем в модель тензор категорий, тензор для текущей буквы в шифровке и скрытое состояние
      output, hidden = model(category_tensor, input_line_tensor[0], hidden)

      # Получаем индекс наиболее вероятной буквы при расшифровке с использованием данного сдвига (категории)
      probas, indices = output.topk(1)

      # Собираем все индексы  -  по каждой букве - в один список
      all_indices.append(indices.flatten().numpy()[0])
  # преобразуем индексы букв в сами буквы и получаем расшифровку
  result = ''.join(INDEX_TO_CHAR[i] for i in all_indices)
  return result


In [127]:
def count_matching_letters(s1, s2):
    count = 0
    for i in range(min(len(s1), len(s2))):
        if s1[i] == s2[i]:
            count += 1
    return count

In [131]:
lines, plains = [], []
for _ in range(10):
  _, line, plain = randomPair(test_cypher_sentences)
  print(line, plain)
  sentence = generate_text(line)
  print(sentence)
  print(count_matching_letters(plain, sentence)/len(plain)*100)

бфэбумвйвнбдуфтзщвзоуабдрйнзбдрпбфрлбтвйждрзппрлбг  ты сказал встречаемся возле вон той раздвоенной б
 ты сказал встречаемся возле вон той раздвоенной б
100.0
кнкбохчрортрдбпвзнуабрузпкнрбнзупкмвбсткбдкжзбуеху или мухоморов наелся осенило лесника при виде сгус
или мухоморов наелся осенило лесника при виде сгус
100.0
нзупкмбсрфтаузппрбпвгн жвнбмвмбмкгртебдэмнвжэдвзфб лесник потрясенно наблюдал как киборг выкладывает 
лесник потрясенно наблюдал как киборг выкладывает 
100.0
срдфрткфюбяфрфбвмтргвфкщзумклбпрозтбнзупкмбпзбухоз повторить этот акробатический номер лесник не суме
повторить этот акробатический номер лесник не суме
100.0
бвевбйвжхощкдрбстрфапхнбжизмбкбдпзйвспрбтвуснэнуаб  ага задумчиво протянул джек и внезапно расплылся 
 ага задумчиво протянул джек и внезапно расплылся 
100.0
бпвжрбгэнрбйвупафюбзербдзтпхфюуабкбурргыкфюбопзбщф  надо было заснять его вернуться и сообщить мне чт
 надо было заснять его вернуться и сообщить мне чт
100.0
бкбщфрбяфрфбфтхгрнвйбгхнюмвзф  и что этот труб

In [110]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

In [117]:
caesar_decrypt(sentence, 2)

'женька категорически не понимал в чем кайф стрельб'